### Import research helpers

In [42]:
from research.research_utils import (
    get_latest_trade_analytics_key,
    expectancy
)


### 1. Must import and reload CondorResearch class
To avoid issues when class is changed
### 2. Setup condor config
### 3. Import the desired object store file


In [3]:
# 1.
import importlib
import research.condor_research as cr
importlib.reload(cr)
CondorResearch = cr.CondorResearch

import research.config as cfg
importlib.reload(cfg)
CondorResearchConfig = cfg.CondorResearchConfig

# 2. 
cfg = CondorResearchConfig()

# 3.
import json
import pandas as pd
qb = QuantBook()

key = 'f0951ccd72cd2566c53b077c8cb50f6b_2024-10-02_00-00-00_trade_analytics.json'

CR = None
df = None
if qb.object_store.contains_key(key):
    string_data = qb.object_store.read(f"{key}")
    json_data = json.loads(string_data)    
    df = pd.json_normalize(json_data)
    CR = CondorResearch(df, cfg)


### Bucket entry time to technical indicator

In [ ]:
CR.entry_exit_stats_group()

out = CR.time_and_adx_stats()
# Where is the strategy profitable?
CR.pivot_view(tidy=out, index="time_bucket", columns = "adx_bucket", values="mean")
# Where do drawdowns come from?
CR.pivot_view(tidy=out, index="time_bucket", columns = "adx_bucket", values="tail_ratio_q05")
CR.time_and_vwap_atr_stats(view="multi")
CR.time_and_gap_pct_stats(view="multi")
CR.time_and_bb_position_stats(view="multi")

### VIX1D indicator shows huge tail risk so investigate why

In [2]:
CR.time_and_vix1d_stats(view="multi")
# woah there is a crazy tail_ratio_q05 in these results 
mask = (
    (CR.df["time_bucket"] == pd.Interval(-0.001, 30.0, closed="right")) &
    (CR.df["vix1d_bucket"] == pd.Interval(9.81, 14.2, closed="right"))
)
sub = CR.df.loc[mask, ["pnl", "pos.position.technicals.current_vix1d", "entry_minutes_since_open"]]
display(sub["pnl"].describe(percentiles=[0.01, 0.05, 0.50, 0.95, 0.99]))
display(sub["pnl"].mean())
display(sub["pnl"].quantile(0.05))
display(sub["pnl"].nsmallest(10))
display(sub["pnl"].value_counts(bins=10).sort_index())

tidy = CR.group_stats(
    keys=["time_bucket","vix1d_bucket","vwap_atr_bucket"],
    min_count=10
)
CR.as_multiindex(tidy, keys=["time_bucket", "vix1d_bucket","vwap_atr_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])



In [ ]:
# check bins
self.df["max_loss_norm"].quantile([0.1,0.25,0.5,0.75,0.9,0.95,0.99])


In [ ]:
tidy = CR.group_stats(["time_bucket", "cushion_norm_bucket"], value_col="pnl")
CR.as_multiindex(tidy, keys=["time_bucket", "cushion_norm_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])


In [5]:
tidy = CR.group_stats(["time_bucket", "max_loss_norm_bucket"], value_col="pnl")
CR.as_multiindex(tidy, keys=["time_bucket", "max_loss_norm_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])

In [6]:
tidy = CR.group_stats(["time_bucket", "em_bucket"], value_col="pnl")
CR.as_multiindex(tidy, keys=["time_bucket", "em_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])

In [ ]:
tidy = CR.group_stats(["time_bucket", "em_bucket", "cushion_norm_bucket"], value_col="pnl")
CR.as_multiindex(tidy, keys=["time_bucket", "em_bucket", "cushion_norm_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])